# MSc Thesis 
# Jacotte Monroe

Code that retrieves automatically the MODIS 8 day scenes for each day of elephant steps. 

## Importing and initializing 

In [1]:
import ee
ee.Authenticate()

True

In [2]:
import geemap
import os
import pandas as pd
import math 

ee.Initialize()

## Set-up

In [3]:
# load Etosha National Park study area
enp = ee.FeatureCollection('WCMC/WDPA/current/polygons') \
        .filter(ee.Filter.eq('ORIG_NAME', 'Etosha'))

# turn ENP study area into a geometry
enp_geom = enp.geometry()

In [4]:
# load extent look up table 
extents_lut = pd.read_csv('data/step_extents/LA2_step_ex_w2027.csv')

In [5]:
# create large extent = region geometry 

large_extent = extents_lut.iloc[-1]

large_extent_coords = [[[large_extent.loc['xmin'], large_extent.loc['ymin']],
                        [large_extent.loc['xmin'], large_extent.loc['ymax']], 
                        [large_extent.loc['xmax'], large_extent.loc['ymin']], 
                        [large_extent.loc['xmax'], large_extent.loc['ymax']]]]

large_region = ee.Geometry.Polygon(large_extent_coords, proj = 'EPSG:32733', evenOdd = False)

In [6]:
# get output folder paths for current MODIS images and images from week prior
# source: https://github.com/gee-community/geemap/blob/master/examples/notebooks/11_export_image.ipynb
out_dir = os.path.join(os.path.expanduser('~'), 'Documents/MSc_Thesis/data/modis_ssf/8_day_2027')

### MODIS Datasets

## Functions

In [7]:
# function to reproject (elephant fixes reprojected from 4326 to 32733 same needs to be done to images) 
def reprojectModis(image):
    return image.reproject('EPSG:32733', None, 250)


In [8]:
# function to clip image to study area
def clipToAOI(image): 
    result = image.clip(bbox)
    return result.copyProperties(image, ['system:id'])

In [9]:
# function to calculate NDVI 
def addNDVI(image): 
    ndvi = image.normalizedDifference(['nir', 'red']).rename('NDVI')
    return image.addBands(ndvi)

## Visualization Parameters (can remove this section when cleaning final code)

In [10]:
# define visualization parameters
visualization_modis = {
  'min': -100.0,
  'max': 8000.0,
  'bands': ['nir', 'nir', 'red'],
}

In [10]:
# initialization and set up of the map
#test = modis_cloudFree.filterDate('2008-10-30', '2008-10-31').median()
#test = test.clip(enp_geom)

Map = geemap.Map()
Map.centerObject(enp_geom)
Map.add_layer(enp_geom, None, 'Etosha National Park')
#Map.add_layer(test, visualization_modis, 'Cloudfree MODIS')
#Map

## Loop

In [11]:
# loop for all table entries 
# 1. get dates
# 2. get coordinates 
# 3. turn into geometry
# 4. get MODIS image
# 5. clip and reproject 
# 6. visualize

for i in range(len(extents_lut)-1):
    # select entry
    extent = extents_lut.iloc[i]
    
    # retrieve extent dates 
    start_date = extent.loc['start_date']
    start_date_prev_week = extent.loc['start_date_prev_week']
    
    # retrieve extent coordinates 
    # source: https://sparkbyexamples.com/pandas/pandas-get-cell-value-from-dataframe/?utm_content=cmp-true
    # source: https://stackoverflow.com/questions/75203044/how-to-create-polygon-from-bbox-data-in-pythonv
    coords = [[[extent.loc['xmin'], extent.loc['ymin']],
              [extent.loc['xmin'], extent.loc['ymax']],
              [extent.loc['xmax'], extent.loc['ymax']],
              [extent.loc['xmax'], extent.loc['ymin']]]]

    # create geometry from coordinates
    bbox = ee.Geometry.Polygon(coords, proj = 'EPSG:32733', evenOdd = False)
    
    # retrieve MODIS 8 day 250m composite image for geometry 
    # source: https://developers.google.com/earth-engine/apidocs/ee-date-advance#colab-python
    modis = ee.ImageCollection('MODIS/061/MOD09GQ') \
        .filterDate(ee.Date(start_date, 'Africa/Maputo').advance(ee.Number(8).multiply(-1), 'day'), ee.Date(start_date, 'Africa/Maputo').advance(ee.Number(1), 'day')) \
        .filterBounds(enp_geom) \
        .select(['sur_refl_b02', 'sur_refl_b01'], ['nir', 'red']) 
    
    modis_prev_week = ee.ImageCollection('MODIS/061/MOD09GQ') \
        .filterDate(ee.Date(start_date_prev_week, 'Africa/Maputo').advance(ee.Number(8).multiply(-1), 'day'), ee.Date(start_date_prev_week, 'Africa/Maputo').advance(ee.Number(1), 'day')) \
        .filterBounds(enp_geom) \
        .select(['sur_refl_b02', 'sur_refl_b01'], ['nir', 'red']) 
    
    # reproject image
    modis_reproj = modis.map(reprojectModis)
    
    modis_prev_week_reproj = modis_prev_week.map(reprojectModis)
    
    # clip image
    modis_clipped = modis_reproj.map(clipToAOI)
    
    modis_prev_week_clipped = modis_prev_week_reproj.map(clipToAOI)

    # calculate NDVI 
    modis_ndvi = modis_clipped.map(addNDVI).select(['NDVI'])

    modis_ndvi_prev_week = modis_prev_week_clipped.map(addNDVI).select(['NDVI'])

    # turn into an image object
    modis_ndvi = modis_ndvi.first()
    modis_ndvi_prev_week = modis_ndvi_prev_week.first()
    
    # map image
    #Map.add_layer(modis_clipped, visualization_modis, 'MODIS image')

    # specify filename 
    filename = os.path.join(out_dir, start_date.replace('-', '_') + '.tif')
    filename_prev_week = os.path.join(out_dir, start_date_prev_week.replace('-', '_') + '.tif')
    
    # export MODIS image to local repository 
    # source: https://github.com/gee-community/geemap/blob/master/examples/notebooks/11_export_image.ipynb
    geemap.ee_export_image(modis_ndvi, filename = filename, region = large_region)
    
    geemap.ee_export_image(modis_ndvi_prev_week, filename = filename_prev_week, region = large_region)
    

Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_Thesis/data/modis_ssf/8_day_2027/2008_11_06.tif
Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_Thesis/data/modis_ssf/8_day_2027/2008_10_30.tif
Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_Thesis/data/modis_ssf/8_day_2027/2008_11_07.tif
Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_Thesis/data/modis_ssf/8_day_2027/2008_10_31.tif
Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_Thesis/data/modis_ssf/8_day_2027/2008_11_08.tif
Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_Thesis/data/modis_ssf/8_day_2027/2008_11_01.tif
Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_Thesis/data/modis_ssf/8_day_2027/2008_11_09.tif
Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_Thesis/d